In [1]:
from pyspark.sql import SparkSession
import pandas as pd 

In [2]:
# 파이썬 설치 위치 지정 
import os

# 경로 지정 
os.environ['PYSPARK_PYTHON'] = '/home/user1/anaconda3/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON']='/home/user1/anaconda3/bin/python3'

In [3]:
# 스파크 객체 생성 
spark = SparkSession.builder.master("local[*]") \
    .enableHiveSupport().appName("hive01") \
    .config("spark.sql.warehouse.dir","/user/hive/warehouse") \
    .config("spark.datasource.hive.metastore.uris","hdfs://192.168.0.19:9000") \
    .getOrCreate()

In [4]:
# 데이터 프레임 생성 

df1 = spark.createDataFrame([(1,'a',10),(2,'b',20),(3,'c',30)]).toDF("id","name","age")
df1.printSchema()
df1.show()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+---+----+---+
| id|name|age|
+---+----+---+
|  1|   a| 10|
|  2|   b| 20|
|  3|   c| 30|
+---+----+---+



### 데이터 프레임을 df1_table로 

In [5]:
df1.createOrReplaceTempView("table1")
spark.sql("SELECT id, name FROM table1").show()

+---+----+
| id|name|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
+---+----+



In [6]:
# 데이터프레임을 pandas로 변경

pd1 = df1.select("*").toPandas()
print(type(pd1))

<class 'pandas.core.frame.DataFrame'>


In [7]:
# 하둡 : DB 생성
spark.sql("create database db05")

AnalysisException: 'org.apache.hadoop.hive.metastore.api.AlreadyExistsException: Database db05 already exists;'

In [8]:
# 데이터 프레임으로 테이블 생성 
spark.sql("create table db05.t01 as select * from table1")

AnalysisException: '`db05`.`t01` already exists.;'

In [9]:
# 데이터 추가하기 
spark.sql("insert into db05.t01 values(4,'a1',40)")

DataFrame[]

In [10]:
spark.sql("select * from db05.t01").show()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|   a| 10|
|  2|   b| 20|
|  3|   c| 30|
|  4|  a1| 40|
+---+----+---+



In [11]:
# DELETE / UPDATE는 hadoop에서 지원하지 않음.